<a href="https://colab.research.google.com/github/HeyHooooooo/SW-/blob/main/03-06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import sqlite3

Python ----> DMBS ------> DB
            SQLITE

package sqlite3 func(SQL)

python -> select -> sqlite db로 data 가져오게 되면 DBMS에 저장되게 되는데 fetch(cursor 에서 obj) 로 가게 되면 python에 저장되게 된다.


In [ ]:
# DBMS - server, Sqlite serverless
# 1. Connection => Client -> Server
# 2. Cursor(작업)는 DBMS가 관리하게 되는데
sqlite3.connect()

In [ ]:
# dbms://id@host:port/databse 로 보통 나오게 되는데 sqlite 는 아이디 계정이 없고 혼잤느느 DB이며 host, 원격지주소가 없지만 파일경로가 필요하다.


In [7]:
conn = sqlite3.connect('test.db.')

In [8]:
type(conn)

sqlite3.Connection

In [9]:
cur = conn.cursor()

In [10]:
type(cur)

sqlite3.Cursor

In [11]:
conn.close()

In [12]:
!ls

sample_data  test.db.


In [13]:
!dir

sample_data  test.db.


In [14]:
conn = sqlite3.connect('test.db')


In [15]:
cur = conn.cursor()

In [ ]:
cur.execute(한번)
cur.execuemany(동일한sql, n번) -> SQL
cur.ececutescript(여러sql) -> 비표준, x

In [ ]:
cur.execute('''
  CREATE TABLE CITY (
    PK INTEGER PRIMARY KEY,
    NAME VARCHAR(1)
  ) ;
  ''')


SQLITE는

In [18]:
cur.execute('''
 INSERT INTO CITY (PK, NAME) VALUES (1, "안암점");
 ''')

In [ ]:
cur.execute('''
  INSERT INTO CITY (NAME) VALUES(?)
  ''')

In [ ]:
cur.exevutemany()

In [22]:
gu = '성북구'
cur.execute('''
  INSERT INTO CITY (NAME) VALUES (?)
  ''', (gu,))

In [23]:
cur.execute('''
  INSERT INTO CITY (NAME) VALUES (:key)
''', {'key':gu})
#Named

In [40]:
cur.executemany('''
 INSERT INTO CITY (NAME) VALUES (?);
 ''', (('노원구',), ('강북구',), ('중구',)))

In [28]:
cur.lastrowid

3

In [29]:
cur.execute('SELECT * FROM CITY')

In [30]:
city = cur.fetchall()


In [31]:
type(city), len(city)

(list, 9)

In [33]:
city[:4]

[(1, '안암점'), (2, '성북구'), (3, '성북구'), (4, '노원구')]

In [35]:
conn.close()

In [36]:
conn = sqlite3.connect('test.db')

In [37]:
cur = conn.cursor()

In [38]:
cur.execute('SELECT * FROM CITY')

In [39]:
cur.fetchall()

[]

In [41]:
conn.commit()

안정성이 확보되어야만 반영을 하는데, commit 은 DBMS를 DB에 반영하는 과정인데, commit이 되는 단위를 transaction이라고 하며,

In [43]:
cur.executescript('''
    DROP TABLE IF EXISTS SUPPLIER;
    CREATE TABLE SUPPLIER (
      PK INTEGER PRIMARY KEY,
      NAME TEXT,
      FK INTEGER NOT NULL
    );
    DROP TABLE IF EXISTS PART;
    CREATE TABLE PART (
      PK INTEGER PRIMARY KEY,
      NAME TEXT
    );
    DROP TABLE IF EXISTS SELLS;
    CREATE TABLE SELLS (
      FK1 INTEGER NOT NULL,
      FK2 INTEGER NOT NULL
    );
   ''')

In [45]:
cur.executemany('''
    INSERT INTO CITY (PK, NAME) VALUES(?,?)
    ''', ((1. '성북구'), (2, '강북구'), (3,'중랑')))

IntegrityError: UNIQUE constraint failed: CITY.PK

In [46]:
cur.executemany('''
 INSERT INTO PART VALUES(:no, :name)
 ''',  ({'no':1, 'name': '아메리카노'},{'no':2, 'name': '카페라떼'},
        {'no':3, 'name': '바닐라라떼'}, {'no':4, 'name': '아이스티'}))

In [53]:
cur.execute('''
  SELECT PK
  FROM CITY
  WHERE NAME LIKE ?
  ORDER BY PK ASC
  LIMIT 0,1
  ''', ('%' + gu + '%',))
rst = cur.fetchone()
fk = 0 if rst == None else rst[0]

In [ ]:
if fk is not None:
  cur.execute('''
    INSERT INTO SUPPLIER VALUES(NULL,?,?)
    ''')

In [58]:
cur.execute('SELECT * FROM SUPPLIER')
cur. fetchall()

[]

In [52]:
cur.execute('''SELECT PK FROM CITY WHERE NAME LIKE "%성북%"
               ORDER BY PK ASC
               LIMIT 0,1
   ''')

cur.fetchone()

중첩 SQL 문법
INSERT INTO SUPPLIER VALUES(NULL,?,(
  SELECT PK FROM CITY
  WHERE NAME LIKE ?
  ORDER BY PK ASC **굵은 텍스트**
  LIMIT 0,1
));

In [61]:
gu = '성북'
cur.execute('''
    SELECT PK
    FROM CITY
    WHERE NAME LIKE ?
    ORDER BY PK ASC
    LIMIT 0, 1
''', ('%'+gu+'%',))
rst = cur.fetchone()
fk = None if rst == None else rst[0]
store = '안암점'
if fk is not None:
    cur.execute('''
        INSERT INTO SUPPLIER VALUES(NULL, ?, ?);
    ''', (store, fk))
cur.execute('SELECT * FROM SUPPLIER')
cur.fetchall()
[(1, '안암점', 1)]
# 중첩 SQL

[(1, '안암점', 1)]

In [65]:
cur.execute('''
    INSERT INTO SUPPLIER VALUES(NULL, ?, (
        SELECT PK
        FROM CITY
        WHERE NAME LIKE ?
        ORDER BY PK ASC
        LIMIT 0, 1
    ));
''', ('종암점', '*성북*'));

IntegrityError: NOT NULL constraint failed: SUPPLIER.FK

In [ ]:
cur,execute('''
 SELECT B.NAME, COUNT(A.NAME) FROM SUPPLIER AS A
 LEFT JOIN CITY AS B
 ON B.PK = A.FK
 GROUP BY B.PK
 ORDER BY B.NAME ASC
 ''')

In [ ]:
#SUPPLIER - PART - SELLS를 만약에 JOIN 한다면

cur.execute('''
    SELECT B.NAME, C.NAME
    FROM SELLS AS A
    INNER JOIN SUPPLIER A B
    ON .PK = A.FK1
    INNER JOIN PART AS C
    ON C.PK = A.FK2)

In [ ]:
cur.execute('''
 SELECT A.NAME, B.FK1
 FROM PART AS A
 LEFT JOIN SELLS AS B
 ON A.PK = B.FK2
 INNER JOIN SUPPLIER AS C
 ON B.FK1 = C.PK
 GROUP BY A.PK )

In [ ]:
cur.execute('''
 SELECT A.NAME, B.FK1
 FROM PART AS A
 LEFT JOIN SELLS AS B
 ON A.PK = B.FK2
 INNER JOIN SUPPLIER AS C
 ON B.FK1 = C.PK
 GROUP BY A.PK )